# Flappy Bird Junco Sim
2D flight sim for EECE5500 Mobile Robotics

In [177]:
############################### Import libraries ###############################
import os
import glob
import time
from datetime import datetime

import matplotlib.pyplot as plt

import numpy as np
from numpy import matlib

import gym
import control

[[2.41421356]]


In [195]:
class Drone():
    m = 3.2
    Iyy = 1/12*m*0.8**2

    S = 0.25
    c = 0.13

    Cl_0 = 0.5
    Cl_alpha = 0.17

    Cd_0 = 0.02
    K = 0.01

    Cm_0 = -0.05
    Cm_alpha = 0.2
    Cm_alpha_dot = -0.1
    Cm_delta_e = 0.5

    g = 9.81

    def __init__(self):
        self.x_thres = 1000
        self.z_thresHigh = 500
        self.z_thresLow = 0
        self.v_thres = 50
        self.theta_thres = np.pi/6
        self.theta_dot_thres = np.pi
        self.gamma_thres = np.pi/6
        self.alpha_dot_stale = 0

        self.limits = np.array([
            self.x_thres,
            self.z_thresHigh,
            self.z_thresLow,
            self.v_thres,
            self.theta_thres,
            self.theta_dot_thres,
            self.gamma_thres
        ])

        self.state = None
        self.prev_action = None
        self.steps_beyond_terminated = None

        self.time = 0.
        self.dt = 0.02

      
    def step(self, action):
        err_msg = f"{action!r} ({type(action)}) invalid"
        assert self.state is not None, "Call reset before using step method."
        
        self.time += self.dt

        #Extra states
        rho = np.random.normal(1.225,0.01)
        alpha = self.state[3] - self.state[4]
        self.q = 0.5*rho*self.state[2]**2
        
        #Control inputs
        thrust, delta_e = action

        # Forces
        Cl = self.Cl_0 + self.Cl_alpha*alpha
        Cd = self.Cd_0 + self.K*Cl**2
        Cm = self.Cm_0 + self.Cm_alpha*alpha + self.Cm_alpha_dot*self.alpha_dot_stale + self.Cm_delta_e*delta_e
        L = self.q*self.S*Cl
        D = self.q*self.S*Cd
        M = self.q*self.S*Cm
        x, z, v, theta, theta_dot, gamma = self.state

        #derivatives
        x_dot = v*np.cos(gamma)
        z_dot = v*np.sin(gamma)
        v_dot = (-D - self.m*self.g*np.sin(gamma) + thrust*np.cos(alpha)) / self.m
        #theta_dot = theta_dot
        theta_ddot = M / self.Iyy
        gamma_dot = (L - self.m*self.g*np.cos(gamma) -thrust*np.sin(alpha)) / (self.m*v)

        #No algebraic loops in my flight code 
        self.alpha_dot_stale = theta_dot - gamma_dot

        # integrate
        x += x_dot*self.dt
        z += z_dot*self.dt
        v += v_dot*self.dt
        theta += theta_dot*self.dt #Removed second order terms.  We don't do that here (Also needs to be 1/2dt^2 f"(t))
        theta_dot += theta_ddot * self.dt
        gamma += gamma_dot*self.dt

        state = np.array([x,z,v,theta,theta_dot,gamma])
        action = np.array([thrust,delta_e])

        self.state = (list(np.reshape(state,(6,))))

        terminated = bool(
            x < -self.x_thres
            or x > self.x_thres
            or z > self.z_thresHigh
            or z < self.z_thresLow
            or v < -self.v_thres
            or v > self.v_thres
            or theta < -self.theta_thres
            or theta > self.theta_thres
            or theta_dot < -self.theta_dot_thres
            or theta_dot > self.theta_dot_thres
            or gamma < -self.gamma_thres
            or gamma > self.gamma_thres
        )

        if not terminated:
            reward = 1.0
        elif self.steps_beyond_terminated is None:
            self.steps_beyond_terminated = 0
            reward = 0.0
        else:
            if self.steps_beyond_terminated == 0:
                print(self.state)
                gym.logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned terminated = True. You "
                    "should always call 'reset()' once you receive 'terminated = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_terminated += 1
            reward = 0.0

        return state, reward, terminated, False

    def reset(self):
        self.state = np.array([0,100,20,0,0,0])
        self.steps_beyond_terminated = None
        self.time = 0

        return self.state

    def close(self):
        pass

In [196]:
env = Drone()
state = env.reset()
action = np.array([1.5,0.1055])
fig, ax = plt.subplots(1,figsize=(10,5))
trajx = []
trajz = []

for t in range(1, 300):
    state, reward, done, _ = env.step(action)
    x = state[0]
    z = state[1]
    v = state[2]
    theta = -state[3]
    theta_dot = -state[4]
    gamma = state[5]
    trajx.append(x)
    trajz.append(z)

ax.plot(trajx, trajz)
fig.show()

False False False False False False False True False False False False


/tmp/ipykernel_41764/2870367086.py:130: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  gym.logger.warn(


In [165]:
#################################### Testing ###################################

env = Drone()
state = env.reset()
action = np.array([18,0.1])

%matplotlib qt
fig, ax = plt.subplots(1,figsize=(10,5))
traj = []

for t in range(1, 1000):
    
    state, reward, done, _ = env.step(action)
    x = state[0]
    z = -state[1]
    v = state[2]
    theta = -state[3]
    theta_dot = -state[4]
    gamma = state[5]
    traj.append((x,z))

    action = np.array([0,0])
        
    if v<25:
        action[0] = 200
    else:
        action[0] = 0
        
    l = 0.8
    plt.plot([x,x+0.25*l*np.cos(theta)],[z,z+0.25*l*np.sin(theta)],'k-')
    plt.plot([x,x-0.75*l*np.cos(theta)],[z,z-0.75*l*np.sin(theta)],'k-')
    plt.xlim(x-20,x+80)
    plt.ylim(z-25,z+25)

    beam = 50
    angle = 60*np.pi/180
    plt.plot([x,x+beam*np.cos(theta+angle/2)],[z,z+beam*np.sin(theta+angle/2)],'k--')
    plt.plot([x,x+beam*np.cos(theta-angle/2)],[z,z+beam*np.sin(theta-angle/2)],'k--')
    xs = x+beam*np.cos(theta+np.linspace(-angle/2,angle/2,10))
    zs = z+beam*np.sin(theta+np.linspace(-angle/2,angle/2,10))
    plt.plot(xs,zs,'k.',markersize=1)

    x,z = zip(*traj)
    plt.plot(x,z,'b-',markersize=1)
    
    plt.show()
    plt.pause(env.dt)

    
    if done:
        break
    else:
        ax.clear()
print(np.around(state,2), reward, done)
env.close()

5.194578716161519 -0.01225412906835599 -0.0011427948926169165
62.06730368696823 -0.007637122511022009 -0.04867114590043324
62.00917718707623 0.060947768015503424 -0.07997074966111917
61.939477390495234 0.12539865178256027 -0.09623033566509959
61.85694744916443 0.18601441258164833 -0.10022620259375872
-0.7562618279353748 0.2330739655651539 -0.09657408649326932
-0.7948721535599685 0.22587802456894937 -0.09620087594223745
-0.8328523486953505 0.21993514301887063 -0.0974150923636109
-0.8780258833305543 0.22184767130932315 -0.09838476649002093
61.53749809356576 0.3128856733838178 -0.0863065833470115
-1.0444404762748305 0.2625835531884493 -0.08743277406440841
-1.1073172837428347 0.2725994505546241 -0.08697255460326944
-1.1468493445921668 0.26052553260085587 -0.08920509956876047
-1.2001802479485284 0.26288175713716055 -0.09055897685652069
-1.2497193705412877 0.26144257440923196 -0.09205908908658658
-1.3100173120129888 0.2696183096406941 -0.09212582013029162
-1.3516832034404742 0.26006480360282